In [42]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [43]:
data = pd.read_csv('data_1.csv')
grade=pd.read_csv('grade.csv')
loan=pd.read_csv('loan_det.csv')

In [54]:
# data.head()
cols=['emp_length','annual_inc','loan_status','dti','inq_last_6mths','open_acc','pub_rec','revol_util','total_acc','total_pymnt','grade','loan_amnt','term','installment']

In [55]:
# loan.head()

In [56]:
data = pd.concat([data, grade], axis=1)
data = pd.concat([data,loan],axis=1)
data = data.loc[:, ~data.columns.duplicated(keep='first')]
data=data[cols]


In [59]:
# merged_X=data.filter(regex=r'^(?!Unnamed).*$')
# merged_X.head()
data=data.dropna()
data.head()

,emp_length,annual_inc,loan_status,dti,inq_last_6mths,open_acc,pub_rec,revol_util,total_acc,total_pymnt,grade,loan_amnt,term,installment
0,10+ years,24000.0,Fully Paid,27.65,1.0,3.0,0.0,83.7,9.0,5861.071414,B,5000,36 months,162.87
1,< 1 year,30000.0,Charged Off,1.00,5.0,3.0,0.0,9.4,4.0,1008.710000,C,2500,60 months,59.83
2,10+ years,12252.0,Fully Paid,8.72,2.0,2.0,0.0,98.5,10.0,3003.653644,C,2400,36 months,84.33
3,10+ years,49200.0,Fully Paid,20.00,1.0,10.0,0.0,21.0,37.0,12226.302210,C,10000,36 months,339.31
4,1 year,80000.0,Current,17.94,0.0,15.0,0.0,53.9,38.0,3242.170000,B,3000,60 months,67.79


In [48]:
# merged_X = merged_X.drop(['mths_since_last_delinq','total_rec_late_fee','pymnt_plan'], axis=1)
# merged_X.head()

,emp_length,annual_inc,loan_status,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_rec_prncp,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_d,grade,loan_amnt,term,int_rate,installment
0,10+ years,24000.0,Fully Paid,27.65,0.0,Jan-85,1.0,3.0,0.0,13648,...,5000.00,861.07,0.00,0.00,Jan-15,B,5000,36 months,10.65,162.87
1,< 1 year,30000.0,Charged Off,1.00,0.0,Apr-99,5.0,3.0,0.0,1687,...,456.46,435.17,117.08,1.11,Apr-13,C,2500,60 months,15.27,59.83
2,10+ years,12252.0,Fully Paid,8.72,0.0,Nov-01,2.0,2.0,0.0,2956,...,2400.00,603.65,0.00,0.00,Jun-14,C,2400,36 months,15.96,84.33
3,10+ years,49200.0,Fully Paid,20.00,0.0,Feb-96,1.0,10.0,0.0,5598,...,10000.00,2209.33,0.00,0.00,Jan-15,C,10000,36 months,13.49,339.31
4,1 year,80000.0,Current,17.94,0.0,Jan-96,0.0,15.0,0.0,27783,...,2233.10,1009.07,0.00,0.00,Jan-16,B,3000,60 months,12.69,67.79


In [60]:
merged_X=data.drop(['loan_status'],axis=1)
# merged_X.head()
Y=data[['loan_status']]
merged_X.head()

,emp_length,annual_inc,dti,inq_last_6mths,open_acc,pub_rec,revol_util,total_acc,total_pymnt,grade,loan_amnt,term,installment
0,10+ years,24000.0,27.65,1.0,3.0,0.0,83.7,9.0,5861.071414,B,5000,36 months,162.87
1,< 1 year,30000.0,1.00,5.0,3.0,0.0,9.4,4.0,1008.710000,C,2500,60 months,59.83
2,10+ years,12252.0,8.72,2.0,2.0,0.0,98.5,10.0,3003.653644,C,2400,36 months,84.33
3,10+ years,49200.0,20.00,1.0,10.0,0.0,21.0,37.0,12226.302210,C,10000,36 months,339.31
4,1 year,80000.0,17.94,0.0,15.0,0.0,53.9,38.0,3242.170000,B,3000,60 months,67.79


In [50]:
# merged_X.describe()

In [61]:
Y.describe()

,loan_status
count,842067
unique,10
top,Current
freq,568447


In [62]:
unique_values = pd.unique(Y['loan_status'])
unique_values

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'],
      dtype=object)

Fully Paid: 90-100% - Borrowers who have fully paid off their loans are considered low risk, so they should be approved for a loan with a high probability.
Charged Off: 0-10% - Charged off loans represent a high risk, so borrowers with this status should be denied a loan with a very high probability.
Current: 80-100% - Current borrowers are meeting their obligations, so they can be considered for a loan with a high probability.
Default: 0-5% - Defaulted loans represent a very high risk, so borrowers with this status should be denied a loan with a very high probability.
Late (31-120 days): 40-70% - Borrowers in this category are late on payments but may still catch up. They represent an increased risk, so they should be considered for a loan with caution.
In Grace Period: 60-80% - Borrowers in the grace period have missed a payment but may catch up. They could be considered for a loan with a moderate probability.
Late (16-30 days): 50-75% - Borrowers in this category are in the early stages of delinquency. They represent an increased risk but may still be considered for a loan.
Does not meet the credit policy. Status:Fully Paid: 70-90% - Borrowers who fully paid but did not meet the credit policy may have some additional risk factors. They should be considered for a loan, but the probability may be lower than for borrowers who fully met the credit policy.
Does not meet the credit policy. Status:Charged Off: 0-10% - Borrowers who did not meet the credit policy and ended up charged off represent a high risk. They should be denied a loan with a very high probability.
Issued: 70-90% - Borrowers with issued loans may still have uncertainty regarding repayment but could be considered for a loan.

In [63]:
from sklearn.preprocessing import LabelEncoder

# Assuming Y is your DataFrame containing the loan status column
# List of loan status categories
loan_status_categories = ['Fully Paid', 'Charged Off', 'Current', 'Default',
                          'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
                          'Does not meet the credit policy. Status:Fully Paid',
                          'Does not meet the credit policy. Status:Charged Off', 'Issued']

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit and transform the loan status column in Y to label encode it
Y['loan_status_encoded'] = label_encoder.fit_transform(Y['loan_status'])
# print("Mapping of categories to encoded numbers:")
encoding_mapping = {category: code for category, code in zip(loan_status_categories, label_encoder.transform(loan_status_categories))}

encoding_mapping


C:\Users\HP\AppData\Local\Temp\ipykernel_9696\1584429750.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['loan_status_encoded'] = label_encoder.fit_transform(Y['loan_status'])


{'Fully Paid': 5,
 'Charged Off': 0,
 'Current': 1,
 'Default': 2,
 'Late (31-120 days)': 9,
 'In Grace Period': 6,
 'Late (16-30 days)': 8,
 'Does not meet the credit policy. Status:Fully Paid': 4,
 'Does not meet the credit policy. Status:Charged Off': 3,
 'Issued': 7}

In [64]:
Y.head()

,loan_status,loan_status_encoded
0,Fully Paid,5
1,Charged Off,0
2,Fully Paid,5
3,Fully Paid,5
4,Current,1


In [ ]:
# merged_X['term']

0          36 months
1          60 months
2          36 months
3          36 months
4          60 months
             ...    
887374     36 months
887375     36 months
887376     60 months
887377     60 months
887378     36 months
Name: term, Length: 887379, dtype: object

In [65]:
import re
# Extract numerical years from 'emp_length' using regex
merged_X['emp_length_years'] = merged_X['emp_length'].apply(lambda x: re.findall(r'\d+', str(x))[0] if pd.notnull(x) else np.nan)

# Convert 'emp_length_years' to numeric
merged_X['emp_length_years'] = pd.to_numeric(merged_X['emp_length_years'], errors='coerce')
merged_X['grade_encoded'] = label_encoder.fit_transform(merged_X['grade'])
merged_X['term_months'] = merged_X['term'].apply(lambda x: int(re.findall(r'\d+', x)[0]))

In [66]:
merged_X.head()

,emp_length,annual_inc,dti,inq_last_6mths,open_acc,pub_rec,revol_util,total_acc,total_pymnt,grade,loan_amnt,term,installment,emp_length_years,grade_encoded,term_months
0,10+ years,24000.0,27.65,1.0,3.0,0.0,83.7,9.0,5861.071414,B,5000,36 months,162.87,10,1,36
1,< 1 year,30000.0,1.00,5.0,3.0,0.0,9.4,4.0,1008.710000,C,2500,60 months,59.83,1,2,60
2,10+ years,12252.0,8.72,2.0,2.0,0.0,98.5,10.0,3003.653644,C,2400,36 months,84.33,10,2,36
3,10+ years,49200.0,20.00,1.0,10.0,0.0,21.0,37.0,12226.302210,C,10000,36 months,339.31,10,2,36
4,1 year,80000.0,17.94,0.0,15.0,0.0,53.9,38.0,3242.170000,B,3000,60 months,67.79,1,1,60


In [71]:
X_Final_cols=['annual_inc','dti','inq_last_6mths','open_acc','total_acc','revol_util','total_pymnt','grade_encoded','loan_amnt','term_months','emp_length_years','installment']
X_Final=merged_X[X_Final_cols]
X_Final.head()

KeyboardInterrupt: 

In [ ]:
# Y_Final=

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [68]:
Y_Final=Y['loan_status_encoded']
Y_Final.head()

0    5
1    0
2    5
3    5
4    1
Name: loan_status_encoded, dtype: int32

In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# Assume X contains your features and y contains your target variable
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_Final, Y_Final, test_size=0.2, random_state=42)

# Drop rows with missing values from the training data
y_train = y_train[X_train.index]  # Make sure to adjust y_train accordingly

# Initialize and fit the RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)

# Evaluate the model on the test data
accuracy = rf_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.8882575082831593


In [78]:
X_train.head()

,annual_inc,dti,inq_last_6mths,open_acc,total_acc,revol_util,total_pymnt,grade_encoded,loan_amnt,term_months,emp_length_years,installment
240180,78000.0,22.69,0.0,12.0,15.0,82.0,7377.60,1,27600,60,7,568.53
770238,51000.0,13.81,2.0,7.0,30.0,37.6,9601.81,5,8975,36,10,352.07
845834,95000.0,19.02,1.0,8.0,16.0,50.4,2479.85,3,6250,36,7,223.27
64475,47500.0,32.43,6.0,10.0,22.0,73.9,8712.80,4,19000,60,2,521.53
440614,45000.0,23.49,0.0,10.0,27.0,66.2,13308.03,1,12000,36,10,398.52


In [70]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()

# Train the classifier
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8882990725236619


In [89]:
# array_pred=[
#     'emp_length',
#     'annual_inc',
#     'dti',
#     'inq_last_6mths',
#     'open_acc',
#     'pub_rec',
#     'revol_util',
#     'total_acc',
#     'total_pymnt',
#     'grade',
#     'loan_amnt',
#     'term',
#     'installment'
# ]
# annual_inc	dti	inq_last_6mths	open_acc	total_acc	revol_util	total_pymnt	grade_encoded	loan_amnt	term_months	emp_length_years	installment
# 'annual_inc','dti','inq_last_6mths','open_acc','total_acc','revol_util','total_pymnt','grade_encoded','loan_amnt','term_months','emp_length_years','installment'
# array_pred1=[250,7,5,6,8,0,0,2,25000000000000,24,0,125000]
# y_pred = model.predict([array_pred1])
# y_pred
# encoding_mapping
# predicted_classes = [encoding_mapping[y_pred] for y_pred in label]

# Display the result
# print(predicted_classes)


array([0], dtype=int64)

In [90]:
import pickle
with open('xgboost_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [83]:
encoding_mapping

{'Fully Paid': 5,
 'Charged Off': 0,
 'Current': 1,
 'Default': 2,
 'Late (31-120 days)': 9,
 'In Grace Period': 6,
 'Late (16-30 days)': 8,
 'Does not meet the credit policy. Status:Fully Paid': 4,
 'Does not meet the credit policy. Status:Charged Off': 3,
 'Issued': 7}